In [2]:
val source_df = spark.read.option("header", "true").option("multiLine", "true").csv("file:/home/nagulraj/tact/datasets/Player_information.csv")

source_df: org.apache.spark.sql.DataFrame = [_c0: string, Player Name: string ... 6 more fields]


In [3]:
source_df.show

+---+--------------------+-------------------+-------------+---------------------+-----------------+------+--------+
|_c0|         Player Name|     Team of Player|National Team|International matches|    Born_date/Age|Height|   Value|
+---+--------------------+-------------------+-------------+---------------------+-----------------+------+--------+
|  0|       Kylian Mbappé|Paris Saint-Germain|       France|                 null|Dec 20, 1998 (23)|  1,78|€160.00m|
|  1|      Erling Haaland|  Borussia Dortmund|       Norway|                   15|Jul 21, 2000 (21)|  1,94|€150.00m|
|  2|     Vinicius Junior|        Real Madrid|       Brazil|                   11|Jul 12, 2000 (21)|  1,76|€100.00m|
|  3|       Mohamed Salah|       Liverpool FC|        Egypt|                   80|Jun 15, 1992 (29)|  1,75|€100.00m|
|  4|          Harry Kane|  Tottenham Hotspur|      England|                   67|Jul 28, 1993 (28)|  1,88|€100.00m|
|  5|       Romelu Lukaku|         Chelsea FC|      Belgium|    

In [4]:
val df1 = source_df.select("National Team")

df1: org.apache.spark.sql.DataFrame = [National Team: string]


In [5]:
df1.show()

+-------------+
|National Team|
+-------------+
|       France|
|       Norway|
|       Brazil|
|        Egypt|
|      England|
|      Belgium|
|     Portugal|
|      Belgium|
|       Brazil|
|      England|
|      England|
|      England|
|      Germany|
|      England|
|        Spain|
|    Argentina|
|      England|
|      England|
|      Senegal|
|  South Korea|
+-------------+
only showing top 20 rows



In [6]:
val df2 = source_df.select("Team of Player")

df2: org.apache.spark.sql.DataFrame = [Team of Player: string]


In [7]:
val merged_df = df2.union(df1)

merged_df: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [Team of Player: string]


In [8]:
val mergedf1= df2.join(df1)
      

mergedf1: org.apache.spark.sql.DataFrame = [Team of Player: string, National Team: string]


In [9]:
mergedf1.show

+-------------------+-------------+
|     Team of Player|National Team|
+-------------------+-------------+
|Paris Saint-Germain|       France|
|Paris Saint-Germain|       Norway|
|Paris Saint-Germain|       Brazil|
|Paris Saint-Germain|        Egypt|
|Paris Saint-Germain|      England|
|Paris Saint-Germain|      Belgium|
|Paris Saint-Germain|     Portugal|
|Paris Saint-Germain|      Belgium|
|Paris Saint-Germain|       Brazil|
|Paris Saint-Germain|      England|
|Paris Saint-Germain|      England|
|Paris Saint-Germain|      England|
|Paris Saint-Germain|      Germany|
|Paris Saint-Germain|      England|
|Paris Saint-Germain|        Spain|
|Paris Saint-Germain|    Argentina|
|Paris Saint-Germain|      England|
|Paris Saint-Germain|      England|
|Paris Saint-Germain|      Senegal|
|Paris Saint-Germain|  South Korea|
+-------------------+-------------+
only showing top 20 rows



In [160]:
merged_df.show()

+-------------------+
|     Team of Player|
+-------------------+
|Paris Saint-Germain|
|  Borussia Dortmund|
|        Real Madrid|
|       Liverpool FC|
|  Tottenham Hotspur|
|         Chelsea FC|
|  Manchester United|
|    Manchester City|
|Paris Saint-Germain|
|    Manchester City|
|  Manchester United|
|  Manchester United|
|      Bayern Munich|
|    Manchester City|
|       FC Barcelona|
|        Inter Milan|
|       Liverpool FC|
|    Manchester City|
|       Liverpool FC|
|  Tottenham Hotspur|
+-------------------+
only showing top 20 rows



In [141]:
 val source_df1 = source_df.drop(col("Born_date/Age"))

source_df1: org.apache.spark.sql.DataFrame = [_c0: string, Player Name: string ... 5 more fields]


In [14]:
val dfAverage = source_df1.agg(avg(source_df1("International matches").cast("int")).as("average"))

<console>: 23: error: not found: value source_df1

In [143]:
dfAverage.show

+------------------+
|           average|
+------------------+
|27.890080428954423|
+------------------+



In [144]:
val avrg=dfAverage.first.getDouble(0)

avrg: Double = 27.890080428954423


In [145]:

val dfnull = source_df.select("International matches")

dfnull: org.apache.spark.sql.DataFrame = [International matches: string]


In [147]:
dfnull.show

+---------------------+
|International matches|
+---------------------+
|                 null|
|                   15|
|                   11|
|                   80|
|                   67|
|                 null|
|                   40|
|                   88|
|                  116|
|                   13|
|                   23|
|                   46|
|                   64|
|                   72|
|                   10|
|                   37|
|                   16|
|                   18|
|                   86|
|                   98|
+---------------------+
only showing top 20 rows



In [148]:
val dfnullfilled  = dfnull.na.fill(avrg,Array("International matches"))


dfnullfilled: org.apache.spark.sql.DataFrame = [International matches: string]


In [149]:
dfnullfilled.show

+---------------------+
|International matches|
+---------------------+
|                 null|
|                   15|
|                   11|
|                   80|
|                   67|
|                 null|
|                   40|
|                   88|
|                  116|
|                   13|
|                   23|
|                   46|
|                   64|
|                   72|
|                   10|
|                   37|
|                   16|
|                   18|
|                   86|
|                   98|
+---------------------+
only showing top 20 rows



In [89]:
dfnull.select(dfnull.columns.map("International matches" => {
    count(when(col("International matches").isNull, true)) as s"${"International matches"}_nulls_count"
  }): _*)
  .show(10)

<console>: 2: error: not a legal formal parameter.

In [150]:
import org.apache.spark.sql.functions.col
val count = dfnull.filter(col("International matches").isNull).count()

import org.apache.spark.sql.functions.col
count: Long = 2


In [151]:
dfnull.filter(isnull($"International matches")).show()


+---------------------+
|International matches|
+---------------------+
|                 null|
|                 null|
+---------------------+



In [152]:
val count_notnull = dfnull.filter(!isnull($"International matches")).count()

count_notnull: Long = 373


In [164]:
import spark.implicits._
val jsonStr = """{ "metadata": { "key": 84896, "value": 54 }}"""
val dfjson = spark.read.json(Seq(jsonStr).toDS)

import spark.implicits._
jsonStr: String = { "metadata": { "key": 84896, "value": 54 }}
dfjson: org.apache.spark.sql.DataFrame = [metadata: struct<key: bigint, value: bigint>]


In [165]:
dfjson.show

+-----------+
|   metadata|
+-----------+
|{84896, 54}|
+-----------+



In [167]:
val rdd = spark.sparkContext.textFile("/home/nagulraj/tact/scala/scala-basics/nagul/abc.txt")

rdd: org.apache.spark.rdd.RDD[String] = /home/nagulraj/tact/scala/scala-basics/nagul/abc.txt MapPartitionsRDD[424] at textFile at <console>:49


In [170]:
val rdd2 = rdd.flatMap(f=>f.split(" "))

rdd2: org.apache.spark.rdd.RDD[String] = MapPartitionsRDD[425] at flatMap at <console>:50


In [172]:
val rdd3 = rdd2.map(m=>(m,1))

rdd3: org.apache.spark.rdd.RDD[(String, Int)] = MapPartitionsRDD[426] at map at <console>:50


In [173]:
val rdd4 = rdd3.filter(a=> a._1.startsWith("a"))

rdd4: org.apache.spark.rdd.RDD[(String, Int)] = MapPartitionsRDD[427] at filter at <console>:50


In [174]:
val rdd5 = rdd4.reduceByKey(_ + _)


rdd5: org.apache.spark.rdd.RDD[(String, Int)] = ShuffledRDD[428] at reduceByKey at <console>:50


In [175]:
val rdd6 = rdd5.map(a=>(a._2,a._1)).sortByKey()
rdd6.foreach(println)

rdd6: org.apache.spark.rdd.RDD[(Int, String)] = ShuffledRDD[432] at sortByKey at <console>:50


In [182]:
rdd5.foreach(println)

In [184]:
 println("Count : "+rdd3.count())

Count : 4
